In [1]:
import pandas as pd
import numpy as np
import pyspark as ps

%matplotlib inline
spark = ps.sql.SparkSession.builder.appName('Test').getOrCreate()

In [ ]:
ls dataset/dataset_csv/

In [18]:
dataset = spark.read.format('csv').options(header='true', inferSchema='true').load('dataset/dataset_csv/current-data-on-the-geographic-distribution-of-covid-19-cases-worldwide.csv')

In [3]:
dataset.show(10)

+-------+---+-----+----+-----+------+-----------------------+-----+--------------------+-----------+------------+
|daterep|day|month|year|cases|deaths|countriesandterritories|geoid|countryterritorycode|popdata2018|continentexp|
+-------+---+-----+----+-----+------+-----------------------+-----+--------------------+-----------+------------+
|   null| 28|    4|2020|  172|     0|            Afghanistan|   AF|                 AFG|   37172386|        Asia|
|   null| 27|    4|2020|   68|    10|            Afghanistan|   AF|                 AFG|   37172386|        Asia|
|   null| 26|    4|2020|  112|     4|            Afghanistan|   AF|                 AFG|   37172386|        Asia|
|   null| 25|    4|2020|   70|     1|            Afghanistan|   AF|                 AFG|   37172386|        Asia|
|   null| 24|    4|2020|  105|     2|            Afghanistan|   AF|                 AFG|   37172386|        Asia|
|   null| 23|    4|2020|   84|     4|            Afghanistan|   AF|                 AFG|

In [9]:
dataset.printSchema()

root
 |-- daterep: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- cases: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- countriesandterritories: string (nullable = true)
 |-- geoid: string (nullable = true)
 |-- countryterritorycode: string (nullable = true)
 |-- popdata2018: integer (nullable = true)
 |-- continentexp: string (nullable = true)



In [ ]:
dataset.select("countriesandterritories").show()

In [ ]:
dataset.filter(df['deaths'] > 100).show()

In [ ]:
dataset.groupBy("countriesandterritories").count().show()

# Transform into Features

In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dataset) \
            for column in list(set(['countriesandterritories','geoid', 'countryterritorycode', 'continentexp'])) ]

pipeline = Pipeline(stages=indexers)
dataset_indexed = pipeline.fit(dataset).transform(dataset)

In [174]:
from pyspark.ml.feature import FeatureHasher
hasher = FeatureHasher(inputCols=['day','month','year',
               'cases', 'countriesandterritories_index',
               'geoid_index', 'countryterritorycode_index', 'popdata2018', 'continentexp_index'],
                       outputCol="features")

featurized = hasher.transform(dataset)
featurized.show(truncate=False)

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:36169)

In [171]:
from pyspark.ml.feature import VectorIndexer
indexer = VectorIndexer(inputCol='features', outputCol="label", maxCategories=10)
indexerModel = indexer.fit(featurized)

KeyboardInterrupt: 

In [ ]:
featurized.show(truncate=False)

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

In [ ]:
training = spark.read.format("libsvm").load("dataset/dataset_csv/sample_libsvm_data.txt")
lrModel = lr.fit(training)

In [ ]:
r1 = Correlation.corr(df, "features").head()